Machine Learning - Homework 1 (due Sep. 13)

Problem 1: Legal reasoning (from Murphy 2.2).

Suppose a crime has been committed. Blood is found at the scene for which there is no innocent explanation. It is of a type which is present in 1% of the population.  The defendant is known to have this rare blood type.

a. The prosecutor claims: “There is a 1% chance that the defendant would have the crime blood type if he
were innocent. Thus there is a 99% chance that he guilty”. This is known as the prosecutor’s fallacy.
What is wrong with this argument?

The prior in this case is that the crime blood type is in 1% of the population. There is a 1% chance of the accused having the crime scene blood type if he is guilty or innocent. 1 - probability of innocence /= probability of guilt.

b. The defender claims: “The crime occurred in a city of 800,000 people. The blood type would be
found in approximately 8000 people. The evidence has provided a probability of just 1 in 8000 that
the defendant is guilty, and thus has no relevance.” This is known as the defender’s fallacy. What is
wrong with this argument (HINT: What happens to the prior in this case if there is *other* evidence presented)?

The argument assumes that the criminal actually came from the city. The criminal could have come from anywhere.

c. Suppose that forensic analysis tells us that that the blood test has 98% sensitivity (true positive rate) and a 1% false positive rate.  Given the information presented in the above two questions, determine the posterior probability the the defendent is guilty, given that the defendent's blood type matches that found at the crime scene *and* that the defendent was one of only 5 people with access to the crime scene *and* that there is no other evidence.  

P(G=1|B=1) = P(B=1|G=1) * P(G=1) / P(B=1) = .249

P(B=1|G=1) = .01
P(G=1) = 1/5
P(B=1) = 98*(1/8000)*(1/5) + .01*(7999/8000)*(4/5) = .00802

Problem 2: Naive Bayes.

One use of the naive Bayes classifier, which is still in practical use today, is as a spam filter.  Consider the corpus of text messages packaged with this homework, which are each labelled as either 'spam' or 'ham'.  In this case, naive Bayes utilizes a Bernoulli model that quantifies the probability of a given word given that the message is either spam or ham.  For example, investigating the text messages here, we find that the word *draw* shows up in spam 27 times, yet in ham only 5.  Thus, we have that
$$ P(X=\mathrm{draw}|Y=\mathrm{ham}) = \frac{5}{5+27}. $$

While this is not particularly strong evidence on its own, we can create a powerful classifier by using the naive assumption in conjunction with all the words in a given message:
$$ P(Y=\mathrm{ham}|X=x) \propto P(Y=\mathrm{ham}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{ham}), $$
$$ P(Y=\mathrm{spam}|X=x) \propto P(Y=\mathrm{spam}) \prod_{i=1}^n P(X_i=x_i|Y=\mathrm{spam}), $$
where $x_i$ are the words in a given message. 

Your task is to write such a classifier.  I have taken the somewhat tedious step of parsing the data for you, yielding the variables *word_dictionary*, which contains the ham and spam counts for each word, as well as *training_labels*, which provides the spam/ham labels for each text message.  I have also parsed a set of test data: *test_messages* is a list, each entry containing another list of the words in the text message, as well as *test_labels* which contains the spam/ham label for each message.

In [1]:
import numpy as np

# Maps from 'ham' or 'spam' strings to zero or one
def mapper(s):
    if s=='spam':
        return 0
    else:
        return 1

# Read in the text file
f = open('SMSSpamCollection','r')
lines = f.readlines()

# Break out the test data
test_lines = lines[:len(lines)//5]
lines = lines[len(lines)//5:]

# Instantiate the frequency dictionary and an array to
# record whether the line is ham or spam
word_dictionary = {}
training_labels = np.zeros(len(lines),dtype=int)

# Loop over all the training messages
for i,l in enumerate(lines):
    # Split into words
    l = l.lower().split()
    # Record the special first word which always ham or spam
    if l[0]=='ham':
        training_labels[i] = 1
    # For each word in the message, record whether the message was ham or spam
    for w in l[1:]:
        # If we've never seen the word before, add a new dictionary entry
        if w not in word_dictionary:
            word_dictionary[w] = [1,1]
        word_dictionary[w][mapper(l[0])] += 1
        
# Loop over the test messages
test_labels = np.zeros(len(test_lines),dtype=int)
test_messages = []
for i,l in enumerate(test_lines):
    l = l.lower().split()
    if l[0]=='ham':
        test_labels[i] = 1
    test_messages.append(l)

Below, I have provided code skeletons.  Your job is to make the code skeletons into an operational naive Bayes spam detector.  (you may discard these skeletons if you would prefer to code this from scratch).  Note that lines where you will need to change the code are marked with a '#!'.

Your first task is train the model:  

In [11]:
#print(word_dictionary)
#What is the prior P(Y=ham) ?
ham_prior = {}
for w in word_dictionary:
    ham_prior[w] = word_dictionary[w][1] / len(word_dictionary)
print(ham_prior)

# What are the class probabilities P(X=word|Y=ham) for each word?
ham_likelihood = {}
for key,val in word_dictionary.items():
    ham_likelihood[key] = None #!

{'no': 0.01587168290225059, "i'm": 0.025109170305676855, 'good': 0.012848505206583809, 'for': 0.03300302317769567, 'the': 0.07465569365132683, 'movie,': 0.00016795431642593214, 'is': 0.04862277460530735, 'it': 0.03014779979845482, 'ok': 0.010665099093046691, 'if': 0.024101444407121263, 'i': 0.14460866644272757, 'leave': 0.003443063486731609, 'in': 0.05282163251595566, 'an': 0.005710446758481693, 'hourish?': 0.00016795431642593214, 'no:)this': 0.00016795431642593214, 'kallis': 0.0003359086328518643, 'home': 0.007725898555592879, 'ground.amla': 0.00016795431642593214, 'town': 0.0015955660060463553, 'durban:)': 0.00016795431642593214, 'so': 0.02645280483708431, 'lets': 0.0011756802149815251, 'make': 0.006130332549546523, 'saturday': 0.0005038629492777964, 'or': 0.015115888478333893, 'monday': 0.0005038629492777964, 'as': 0.009321464561639235, 'per': 0.0007557944239166947, 'convenience.': 0.00016795431642593214, 'hey...': 0.0004198857910648304, 'what': 0.015787705744037622, 'time': 0.01016

Your next task is to make predictions on a set of test examples which were held back from the training procedure (see *test_messages* variable).  For each of these messages, compute the ham and spam probabilities.

In [ ]:
# Where to hold the ham and spam posteriors
posteriors = np.zeros((len(test_lines),2))

# Loop over all the messages in the test set
for i,m in enumerate(test_messages):
    posterior_ham = 1.0 
    posterior_spam = 1.0 
    #! Don't forget to include the prior!
    # Loop over all the words in each message
    for w in m:
        # #! What is the purpose of this try/except handler?
        try:
            posterior_ham *= 1 #!
            posterior_spam *= 1 #!
        except KeyError:
            pass
    
    # Notice the normalization factor (denominator) 
    # to turn these into proper probabilities!
    posteriors[i,0] = posterior_spam/(posterior_spam + posterior_ham)
    posteriors[i,1] = posterior_ham/(posterior_spam + posterior_ham)

Finally, make a ham/spam prediction based on your posterior probabilities.  Compare these to the labels contained in test_labels.  Report the accuracy of your classifier as percentage correct.

In [ ]:
#!